In [1]:

import sys
sys.path.insert(1, osp.join(os.pardir, 'py'))
from choropleth_utils import ChoroplethUtilities
from stats_scraping_utils import StatsScrapingUtilities
from storage import Storage
import numpy as np
import os
import re

s = Storage()
ssu = StatsScrapingUtilities(s=s)

In [2]:

saves_svg_folder = os.path.abspath('../saves/svg')
# !start %windir%\explorer.exe "{saves_svg_folder}"

In [8]:

eurasia_df.sample(5).T

,80,30,58,63,8
country_name,Korea (Democratic People's Republic of),France,Kyrgyzstan,Nepal,Estonia
country_region,NaN,Europe,Asia,Asia,Europe
imf_estimate,NaN,2940428.0,8150.0,34265.0,36039.0
imf_year,NaN,2021.0,2021.0,2021.0,2021.0
un_estimate,NaN,2630318.0,7736.0,33079.0,30650.0
...,...,...,...,...,...
short_form_name,NKO,FRA,KYR,NEP,EST
pwrindx_score,0.4621,0.1283,2.2011,3.4016,2.6527
city_equivalent_gdp,Honolulu,New York,Honolulu,Honolulu,Honolulu
centroid_id,path-korea-democratic-people-s-republic-of,France_mainland,path-kyrgyzstan,path-nepal,Estonia_mainland


In [23]:

eurasia_df = s.load_object('eurasia_df')
if 'centroid_id' not in eurasia_df.columns:
    import svgpathtools
    c = ChoroplethUtilities(s=s)
    
    # Get the paths that are associated with all the contries in our eurasia dataset
    codes_list = []
    df = eurasia_df.set_index('country_name', drop=True).rename(columns={'country_code': 'district_abbreviation'})
    for district_name, row_series in df.sort_index(axis='index', ascending=False).iterrows():
        text_id = c.indexize_string(district_name)
        codes_list.append(f'district-{text_id}')
    file_path = os.path.join(saves_svg_folder, 'AF_state_equivalent_save_a_copy.svg')
    page_soup = ssu.get_page_soup(file_path, verbose=True)
    path_soups_list = page_soup.find_all('path', id=re.compile('^(' + '|'.join(codes_list) + ')$'))
    
    for path_soup in path_soups_list:
        
        # Find the path object with the greatest area
        path_id = path_soup['id']
        path_obj = svgpathtools.parse_path(pathdef=path_soup['d'], current_pos=0j)
        if path_obj is None:
            text_x = np.nan
            text_y = np.nan
            centroid_id = np.nan
            centroid_x = np.nan
            centroid_y = np.nan
        else:
            poly_obj = c.path_to_poly(path_obj)
            text_x = poly_obj.centroid.x
            text_y = poly_obj.centroid.y
            centroid_id = path_id
            centroid_x = poly_obj.centroid.x
            centroid_y = poly_obj.centroid.y

        # Associate the centroids with the country code
        mask_series = (path_id == eurasia_df.country_name.map(lambda x: f'district-{c.indexize_string(x)}'))
        eurasia_df.loc[mask_series, 'text_x'] = text_x
        eurasia_df.loc[mask_series, 'text_y'] = text_y
        eurasia_df.loc[mask_series, 'centroid_id'] = centroid_id
        eurasia_df.loc[mask_series, 'centroid_x'] = centroid_x
        eurasia_df.loc[mask_series, 'centroid_y'] = centroid_y
    s.store_objects(eurasia_df=eurasia_df)

In [28]:

eurasia_df = s.load_object('eurasia_df')
if 'dy' not in eurasia_df.columns:
    world_map_df = s.load_object('world_map_df')
    dy_dict = world_map_df.set_index('country_name', drop=True).dy.to_dict()
    eurasia_df['dy'] = eurasia_df.country_name.map(lambda x: dy_dict.get(x, 9))
    s.store_objects(eurasia_df=eurasia_df)

In [107]:

mask_series = (eurasia_df.pwrindx_score < 100)
one_country_df = eurasia_df[mask_series].set_index('country_name',
                                                   drop=True).rename(columns={'country_code': 'district_abbreviation'})
df = eurasia_df[mask_series].set_index('country_code', drop=True)
c = ChoroplethUtilities(iso_3166_2_code='af', one_country_df=one_country_df, all_countries_df=df, s=s)
c.create_label_line_file()
one_country_df['log_pwrindx_score'] = one_country_df.pwrindx_score.map(lambda x: 3 + np.log(x))
numeric_column_name = 'log_pwrindx_score'
string_column_name = 'wikipedia_title'
svg_file_path = c.create_country_colored_labeled_map(numeric_column_name=numeric_column_name,
                                                     string_column_name=string_column_name,
                                                     one_country_df=one_country_df, cmap='viridis_r')
print(os.path.abspath(svg_file_path))

C:\Users\daveb\OneDrive\Documents\GitHub\StatsByCountry\saves\svg\AF_log_pwrindx_score_wikipedia_title.svg


In [106]:

mask_series = (eurasia_df.country_name == 'Belarus')
# eurasia_df.loc[mask_series, 'font_size'] = 5
eurasia_df.loc[mask_series, 'text_x'] = 462.5936
eurasia_df.loc[mask_series, 'text_y'] = 323.29926
eurasia_df.loc[mask_series, 'dy'] = 0.0
s.store_objects(eurasia_df=eurasia_df)
columns_list = ['country_name', 'wikipedia_title', 'font_size', 'text_x', 'text_y', 'dy']
eurasia_df[mask_series][columns_list].to_dict('records')[0]

Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\StatsByCountry\saves\pkl\eurasia_df.pkl


{'country_name': 'Belarus',
 'wikipedia_title': 'Belarus',
 'font_size': 12.0,
 'text_x': 462.5936,
 'text_y': 323.29926,
 'dy': 0.0}